# Result Analysis

In [1]:
import os
import cv2
import numpy as np
import lpips
import torch
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
imgs_dir = os.path.join(os.getcwd(), 'data', 'bob_clarens')
number_of_imgs = 200

In [3]:
def compute_fft(image):
    # Apply FFT
    f = np.fft.fft2(image)
    # Shift the zero frequency component to the center
    fshift = np.fft.fftshift(f)
    # Compute the magnitude spectrum
    magnitude_spectrum = np.abs(fshift)
    return magnitude_spectrum.astype(np.float32)

# MAE

In [4]:
def calculate_mae(image1, image2):
    # Ensure the images have the same dimensions
    assert image1.shape == image2.shape, "Images must have the same dimensions"
    
    # Compute the Mean Absolute Error
    mae = np.mean(np.abs(image1 - image2))
    return mae

## Image Domain

In [5]:
mae_0_lst = np.array([])
mae_1_lst = np.array([])
mae_2_lst = np.array([])
mae_3_lst = np.array([])
mae_4_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as RGB
    ref = np.array(Image.open(os.path.join(imgs_dir, 'ref', f'r_{i}.png')).convert('RGB'))
    gsir = np.array(Image.open(os.path.join(imgs_dir, 'gsir', f'r_{i}.png')).convert('RGB'))
    tensoir = np.array(Image.open(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png')).convert('RGB'))
    gtv = np.array(Image.open(os.path.join(imgs_dir, 'mine_gt', f'r_{i}.png')).convert('RGB'))
    mine = np.array(Image.open(os.path.join(imgs_dir, 'mine', f'r_{i}.png')).convert('RGB'))

    mae_0 = calculate_mae(ref, ref)
    mae_1 = calculate_mae(ref, gsir)
    mae_2 = calculate_mae(ref, tensoir)
    mae_3 = calculate_mae(ref, gtv)
    mae_4 = calculate_mae(ref, mine)

    mae_0_lst = np.append(mae_0_lst, mae_0)
    mae_1_lst = np.append(mae_1_lst, mae_1)
    mae_2_lst = np.append(mae_2_lst, mae_2)
    mae_3_lst = np.append(mae_3_lst, mae_3)
    mae_4_lst = np.append(mae_4_lst, mae_4)
        
print(f'Sanity Check: {mae_0_lst.mean()}')
print(f'MAE between Reference and GS-IR: {mae_1_lst.mean()}')
print(f'MAE between Reference and TensoIR: {mae_2_lst.mean()}')
print(f'MAE between Reference and GTV: {mae_3_lst.mean()}')
print(f'MAE between Reference and Mine: {mae_4_lst.mean()}')

Sanity Check: 0.0
MAE between Reference and GS-IR: 19.7776188203125
MAE between Reference and TensoIR: 24.505929799479162
MAE between Reference and GTV: 16.144162507812503
MAE between Reference and Mine: 20.894783458333333


## Fourier Domain

In [6]:
mae_0_lst = np.array([])
mae_1_lst = np.array([])
mae_2_lst = np.array([])
mae_3_lst = np.array([])
mae_4_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as grayscale
    ref_gray = cv2.imread(os.path.join(imgs_dir, 'ref', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gsir_gray = cv2.imread(os.path.join(imgs_dir, 'gsir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    tensoir_gray = cv2.imread(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gtv_gray = cv2.imread(os.path.join(imgs_dir, 'mine_gt', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    mine_gray = cv2.imread(os.path.join(imgs_dir, 'mine', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)

    ref_spectrum = compute_fft(ref_gray)
    gsir_spectrum = compute_fft(gsir_gray)
    tensoir_spectrum = compute_fft(tensoir_gray)
    gtv_spectrum = compute_fft(gtv_gray)
    mine_spectrum = compute_fft(mine_gray)

    mae_0 = calculate_mae(ref_spectrum, ref_spectrum)
    mae_1 = calculate_mae(ref, gsir)
    mae_2 = calculate_mae(ref, tensoir)
    mae_3 = calculate_mae(ref, gtv)
    mae_4 = calculate_mae(ref, mine)

    mae_0_lst = np.append(mae_0_lst, mae_0)
    mae_1_lst = np.append(mae_1_lst, mae_1)
    mae_2_lst = np.append(mae_2_lst, mae_2)
    mae_3_lst = np.append(mae_3_lst, mae_3)
    mae_4_lst = np.append(mae_4_lst, mae_4)

print(f'Sanity Check: {mae_0_lst.mean()}')
print(f'MAE between Reference and GS-IR: {mae_1_lst.mean()}')
print(f'MAE between Reference and TensoIR: {mae_2_lst.mean()}')
print(f'MAE between Reference and GTV: {mae_3_lst.mean()}')
print(f'MAE between Reference and Mine: {mae_4_lst.mean()}')

Sanity Check: 0.0
MAE between Reference and GS-IR: 23.853976041666666
MAE between Reference and TensoIR: 25.565108333333338
MAE between Reference and GTV: 17.261838541666666
MAE between Reference and Mine: 23.56360260416667


# MSE

In [7]:
def mse(imageA, imageB):
    # Compute the Mean Squared Error between the two images
    return np.mean((imageA - imageB) ** 2)

## Image Domain

In [8]:
mse_0_lst = np.array([])
mse_1_lst = np.array([])
mse_2_lst = np.array([])
mse_3_lst = np.array([])
mse_4_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as RGB
    ref = np.array(Image.open(os.path.join(imgs_dir, 'ref', f'r_{i}.png')).convert('RGB'))
    gsir = np.array(Image.open(os.path.join(imgs_dir, 'gsir', f'r_{i}.png')).convert('RGB'))
    tensoir = np.array(Image.open(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png')).convert('RGB'))
    gtv = np.array(Image.open(os.path.join(imgs_dir, 'mine_gt', f'r_{i}.png')).convert('RGB'))
    mine = np.array(Image.open(os.path.join(imgs_dir, 'mine', f'r_{i}.png')).convert('RGB'))

    mse_0 = mse(ref, ref)
    mse_1 = mse(ref, gsir)
    mse_2 = mse(ref, tensoir)
    mse_3 = mse(ref, gtv)
    mse_4 = mse(ref, mine)

    mse_0_lst = np.append(mse_0_lst, mse_0)
    mse_1_lst = np.append(mse_1_lst, mse_1)
    mse_2_lst = np.append(mse_2_lst, mse_2)
    mse_3_lst = np.append(mse_3_lst, mse_3)
    mse_4_lst = np.append(mse_4_lst, mse_4)

print(f'Sanity Check: {mse_0_lst.mean()}')
print(f'MSE between Reference and GS-IR: {mse_1_lst.mean()}')
print(f'MSE between Reference and TensoIR: {mse_2_lst.mean()}')
print(f'MSE between Reference and GTV: {mse_3_lst.mean()}')
print(f'MSE between Reference and Mine: {mse_4_lst.mean()}')

Sanity Check: 0.0
MSE between Reference and GS-IR: 4.6380554973958334
MSE between Reference and TensoIR: 8.323264820312499
MSE between Reference and GTV: 5.023929658854167
MSE between Reference and Mine: 5.028422442708333


## Fourier Domain

In [9]:
# Compute MSE between the reference spectrum and each reconstruction spectrum
mse_0_lst = np.array([])
mse_1_lst = np.array([])
mse_2_lst = np.array([])
mse_3_lst = np.array([])
mse_4_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as grayscale
    ref_gray = cv2.imread(os.path.join(imgs_dir, 'ref', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gsir_gray = cv2.imread(os.path.join(imgs_dir, 'gsir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    tensoir_gray = cv2.imread(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gtv_gray = cv2.imread(os.path.join(imgs_dir, 'mine_gt', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    mine_gray = cv2.imread(os.path.join(imgs_dir, 'mine', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)

    ref_spectrum = compute_fft(ref_gray)
    gsir_spectrum = compute_fft(gsir_gray)
    tensoir_spectrum = compute_fft(tensoir_gray)
    gtv_spectrum = compute_fft(gtv_gray)
    mine_spectrum = compute_fft(mine_gray)

    mse_0 = mse(ref_spectrum, ref_spectrum)
    mse_1 = mse(ref_spectrum, gsir_spectrum)
    mse_2 = mse(ref_spectrum, tensoir_spectrum)
    mse_3 = mse(ref_spectrum, gtv_spectrum)
    mse_4 = mse(ref_spectrum, mine_spectrum)

    mse_0_lst = np.append(mse_0_lst, mse_0)
    mse_1_lst = np.append(mse_1_lst, mse_1)
    mse_2_lst = np.append(mse_2_lst, mse_2)
    mse_3_lst = np.append(mse_3_lst, mse_3)
    mse_4_lst = np.append(mse_4_lst, mse_4)

print(f'Sanity Check: {mse_0_lst.mean()}')
print('MSE between Reference and GS-IR:', mse_1_lst.mean())
print('MSE between Reference and TensoIR:', mse_2_lst.mean())
print('MSE between Reference and GTV:', mse_3_lst.mean())
print('MSE between Reference and Mine:', mse_4_lst.mean())

Sanity Check: 0.0
MSE between Reference and GS-IR: 54801073.13
MSE between Reference and TensoIR: 43108284.6425
MSE between Reference and GTV: 38507604.7
MSE between Reference and Mine: 52868310.805


# SSIM

## Image Domain

In [10]:
ssim_score0_lst = np.array([])
ssim_score1_lst = np.array([])
ssim_score2_lst = np.array([])
ssim_score3_lst = np.array([])
ssim_score4_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as RGB
    ref = np.array(Image.open(os.path.join(imgs_dir, 'ref', f'r_{i}.png')).convert('RGB'))
    gsir = np.array(Image.open(os.path.join(imgs_dir, 'gsir', f'r_{i}.png')).convert('RGB'))
    tensoir = np.array(Image.open(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png')).convert('RGB'))
    gtv = np.array(Image.open(os.path.join(imgs_dir, 'mine_gt', f'r_{i}.png')).convert('RGB'))
    mine = np.array(Image.open(os.path.join(imgs_dir, 'mine', f'r_{i}.png')).convert('RGB'))

    data_range = 255.0
    ssim_score0, ssim_image0 = ssim(ref, ref, data_range=data_range, full=True, win_size=11, channel_axis=2)
    ssim_score1, ssim_image1 = ssim(ref, gsir, data_range=data_range, full=True, win_size=11, channel_axis=2)
    ssim_score2, ssim_image2 = ssim(ref, tensoir, data_range=data_range, full=True, win_size=11, channel_axis=2)
    ssim_score3, ssim_image3 = ssim(ref, gtv, data_range=data_range, full=True, win_size=11, channel_axis=2)
    ssim_score4, ssim_image4 = ssim(ref, mine, data_range=data_range, full=True, win_size=11, channel_axis=2)

    ssim_score0_lst = np.append(ssim_score0_lst, ssim_score0)
    ssim_score1_lst = np.append(ssim_score1_lst, ssim_score1)
    ssim_score2_lst = np.append(ssim_score2_lst, ssim_score2)
    ssim_score3_lst = np.append(ssim_score3_lst, ssim_score3)
    ssim_score4_lst = np.append(ssim_score4_lst, ssim_score4)

print(f'Sanity Check: {ssim_score0_lst.mean()}')
print(f'SSIM between Reference and GS-IR: {ssim_score1_lst.mean()}')
print(f'SSIM between Reference and TensoIR: {ssim_score2_lst.mean()}')
print(f'SSIM between Reference and GTV: {ssim_score3_lst.mean()}')
print(f'SSIM between Reference and Mine: {ssim_score4_lst.mean()}')

Sanity Check: 1.0
SSIM between Reference and GS-IR: 0.913605295248951
SSIM between Reference and TensoIR: 0.914796172882652
SSIM between Reference and GTV: 0.9031371764912148
SSIM between Reference and Mine: 0.8974539038191308


## Fourier Domain

In [11]:
ssim_score0_lst = np.array([])
ssim_score1_lst = np.array([])
ssim_score2_lst = np.array([])
ssim_score3_lst = np.array([])
ssim_score4_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as grayscale
    ref_gray = cv2.imread(os.path.join(imgs_dir, 'ref', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gsir_gray = cv2.imread(os.path.join(imgs_dir, 'gsir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    tensoir_gray = cv2.imread(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gtv_gray = cv2.imread(os.path.join(imgs_dir, 'mine_gt', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    mine_gray = cv2.imread(os.path.join(imgs_dir, 'mine', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)

    ref_spectrum = compute_fft(ref_gray)
    gsir_spectrum = compute_fft(gsir_gray)
    tensoir_spectrum = compute_fft(tensoir_gray)
    gtv_spectrum = compute_fft(gtv_gray)
    mine_spectrum = compute_fft(mine_gray)

    # Compute SSIM
    data_range_ref = ref_spectrum.max() - ref_spectrum.min()
    data_range_gsir = gsir_spectrum.max() - gsir_spectrum.min()
    data_range_tensoir = tensoir_spectrum.max() - tensoir_spectrum.min()
    data_range_gtv = gtv_spectrum.max() - gtv_spectrum.min()
    data_range_mine = mine_spectrum.max() - mine_spectrum.min()

    ssim_score0, ssim_image0 = ssim(ref_spectrum, ref_spectrum, full=True, data_range=data_range_ref)
    ssim_score1, ssim_image1 = ssim(ref_spectrum, gsir_spectrum, full=True, data_range=max(data_range_gsir, data_range_ref))
    ssim_score2, ssim_image2 = ssim(ref_spectrum, tensoir_spectrum, full=True, data_range=max(data_range_tensoir, data_range_ref))
    ssim_score3, ssim_image3 = ssim(ref_spectrum, gtv_spectrum, full=True, data_range=max(data_range_gtv, data_range_ref))
    ssim_score4, ssim_image4 = ssim(ref_spectrum, mine_spectrum, full=True, data_range=max(data_range_mine, data_range_ref))

    ssim_score0_lst = np.append(ssim_score0_lst, ssim_score0)
    ssim_score1_lst = np.append(ssim_score1_lst, ssim_score1)
    ssim_score2_lst = np.append(ssim_score2_lst, ssim_score2)
    ssim_score3_lst = np.append(ssim_score3_lst, ssim_score3)
    ssim_score4_lst = np.append(ssim_score4_lst, ssim_score4)

print(f'Sanity Check: {ssim_score0_lst.mean()}')
print('SSIM between Reference and GS-IR:', ssim_score1_lst.mean())
print('SSIM between Reference and TensoIR:', ssim_score2.mean())
print('SSIM between Reference and Mine:', ssim_score3.mean())
print('SSIM between Reference and Mine:', ssim_score4.mean()) 

Sanity Check: 1.0
SSIM between Reference and GS-IR: 0.9982483930688475
SSIM between Reference and TensoIR: 0.9988099948507846
SSIM between Reference and Mine: 0.9995848241336669
SSIM between Reference and Mine: 0.998937512102088


# PSNR

## Image Domain

In [12]:
psnr_score0_lst = np.array([])
psnr_score1_lst = np.array([])
psnr_score2_lst = np.array([])
psnr_score3_lst = np.array([])
psnr_score4_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as RGB
    ref = np.array(Image.open(os.path.join(imgs_dir, 'ref', f'r_{i}.png')).convert('RGB'))
    gsir = np.array(Image.open(os.path.join(imgs_dir, 'gsir', f'r_{i}.png')).convert('RGB'))
    tensoir = np.array(Image.open(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png')).convert('RGB'))
    gtv = np.array(Image.open(os.path.join(imgs_dir, 'mine_gt', f'r_{i}.png')).convert('RGB'))
    mine = np.array(Image.open(os.path.join(imgs_dir, 'mine', f'r_{i}.png')).convert('RGB'))

    data_range = 255.0
    psnr_score0 = psnr(ref, ref, data_range=data_range)
    psnr_score1 = psnr(ref, gsir, data_range=data_range)
    psnr_score2 = psnr(ref, tensoir, data_range=data_range)
    psnr_score3 = psnr(ref, gtv, data_range=data_range)
    psnr_score4 = psnr(ref, mine, data_range=data_range)

    psnr_score0_lst = np.append(psnr_score0_lst, psnr_score0)
    psnr_score1_lst = np.append(psnr_score1_lst, psnr_score1)
    psnr_score2_lst = np.append(psnr_score2_lst, psnr_score2)
    psnr_score3_lst = np.append(psnr_score3_lst, psnr_score3)
    psnr_score4_lst = np.append(psnr_score4_lst, psnr_score4)

print(f'Sanity Check: {psnr_score0_lst.mean()}')
print(f'PSNR between Reference and GS-IR: {psnr_score1_lst.mean()}')
print(f'PSNR between Reference and TensoIR: {psnr_score2_lst.mean()}')
print(f'PSNR between Reference and GTV: {psnr_score3_lst.mean()}')
print(f'PSNR between Reference and Mine: {psnr_score4_lst.mean()}')

/Users/paulina/opt/anaconda3/lib/python3.9/site-packages/skimage/metrics/simple_metrics.py:163: RuntimeWarning: divide by zero encountered in double_scalars
  return 10 * np.log10((data_range ** 2) / err)


Sanity Check: inf
PSNR between Reference and GS-IR: 27.31194726812064
PSNR between Reference and TensoIR: 27.883475167862233
PSNR between Reference and GTV: 25.412697783204422
PSNR between Reference and Mine: 25.225169787105393


## Fourier Domain

In [13]:
psnr_score0_lst = np.array([])
psnr_score1_lst = np.array([])
psnr_score2_lst = np.array([])
psnr_score3_lst = np.array([])
psnr_score4_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as grayscale
    ref_gray = cv2.imread(os.path.join(imgs_dir, 'ref', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gsir_gray = cv2.imread(os.path.join(imgs_dir, 'gsir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    tensoir_gray = cv2.imread(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gtv_gray = cv2.imread(os.path.join(imgs_dir, 'mine_gt', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    mine_gray = cv2.imread(os.path.join(imgs_dir, 'mine', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)

    ref_spectrum = compute_fft(ref_gray)
    gsir_spectrum = compute_fft(gsir_gray)
    tensoir_spectrum = compute_fft(tensoir_gray)
    gtv_spectrum = compute_fft(gtv_gray)
    mine_spectrum = compute_fft(mine_gray)

    data_range_ref = ref_spectrum.max() - ref_spectrum.min()
    data_range_gsir = gsir_spectrum.max() - gsir_spectrum.min()
    data_range_tensoir = tensoir_spectrum.max() - tensoir_spectrum.min()
    data_range_gtv = gtv_spectrum.max() - gtv_spectrum.min()
    data_range_mine = mine_spectrum.max() - mine_spectrum.min()

    psnr_value0 = psnr(ref_spectrum, ref_spectrum, data_range=data_range_ref)
    psnr_value1 = psnr(ref_spectrum, gsir_spectrum, data_range=max(data_range_gsir, data_range_ref))
    psnr_value2 = psnr(ref_spectrum, tensoir_spectrum, data_range=max(data_range_tensoir, data_range_ref))
    psnr_value3 = psnr(ref_spectrum, gtv_spectrum, data_range=max(data_range_gtv, data_range_ref))
    psnr_value4 = psnr(ref_spectrum, mine_spectrum, data_range=max(data_range_mine, data_range_ref))

    psnr_score0_lst = np.append(psnr_score0_lst, psnr_value0)
    psnr_score1_lst = np.append(psnr_score1_lst, psnr_value1)
    psnr_score2_lst = np.append(psnr_score2_lst, psnr_value2)
    psnr_score3_lst = np.append(psnr_score3_lst, psnr_value3)
    psnr_score4_lst = np.append(psnr_score4_lst, psnr_value4)

print(f'Sanity Check: {psnr_score0_lst.mean()}')
print('PSNR between Reference and GS-IR:', psnr_score1_lst.mean())
print('PSNR between Reference and TensoIR:', psnr_score2_lst.mean())
print('PSNR between Reference and GTV:', psnr_score3_lst.mean())
print('PSNR between Reference and Mine:', psnr_score4_lst.mean())

Sanity Check: inf
PSNR between Reference and GS-IR: 64.72241492603902
PSNR between Reference and TensoIR: 65.81585578852078
PSNR between Reference and GTV: 66.15547068157515
PSNR between Reference and Mine: 64.55433119670506


# LPIPS

In [14]:
# Initialize LPIPS model
# You can use 'alex', 'vgg', or 'squeeze'
lpips_model_alex = lpips.LPIPS(net='alex') 
lpips_model_vgg = lpips.LPIPS(net='vgg') 
lpips_model_squeeze = lpips.LPIPS(net='squeeze') 

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/Users/paulina/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/paulina/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /Users/paulina/opt/anaconda3/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/Users/paulina/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /Users/paulina/opt/anaconda3/lib/python3.9/site-packages/lpips/weights/v0.1/vgg.pth
Setting up [LPIPS] perceptual loss: trunk [squeeze], v[0.1], spatial [off]
Loading model from: /Users/paulina/opt/anaconda3/lib/python3.9/site-packages/lpips/weights/v0.1/squeeze.pth


/Users/paulina/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Image Domain

In [15]:
def normalise_rgb_for_lpips(image):
    # Normalise the image to the range [-1, 1], as expected by LPIPS
    return (2 * (image / 255.0) - 1.0).astype(np.float32)

In [16]:
ref_imgs_rgb = []
gsir_imgs_rgb = []
tensoir_imgs_rgb = []
gtv_imgs_rgb = []
mine_imgs_rgb = []

for i in range(number_of_imgs):
    print(f'Processing image {i}...')
    # Load the images as RGB
    ref = np.array(Image.open(os.path.join(imgs_dir, 'ref', f'r_{i}.png')).convert('RGB'))
    gsir = np.array(Image.open(os.path.join(imgs_dir, 'gsir', f'r_{i}.png')).convert('RGB'))
    tensoir = np.array(Image.open(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png')).convert('RGB'))
    gtv = np.array(Image.open(os.path.join(imgs_dir, 'mine_gt', f'r_{i}.png')).convert('RGB'))
    mine = np.array(Image.open(os.path.join(imgs_dir, 'mine', f'r_{i}.png')).convert('RGB'))

    ref_tensor = torch.from_numpy(normalise_rgb_for_lpips(ref)).permute(2, 0, 1).unsqueeze(0)
    gsir_tensor = torch.from_numpy(normalise_rgb_for_lpips(gsir)).permute(2, 0, 1).unsqueeze(0)
    tensoir_tensor = torch.from_numpy(normalise_rgb_for_lpips(tensoir)).permute(2, 0, 1).unsqueeze(0)
    gtv_tensor = torch.from_numpy(normalise_rgb_for_lpips(gtv)).permute(2, 0, 1).unsqueeze(0)
    mine_tensor = torch.from_numpy(normalise_rgb_for_lpips(mine)).permute(2, 0, 1).unsqueeze(0)

    ref_imgs_rgb.append(ref_tensor)
    gsir_imgs_rgb.append(gsir_tensor)
    tensoir_imgs_rgb.append(tensoir_tensor)
    gtv_imgs_rgb.append(gtv_tensor)
    mine_imgs_rgb.append(mine_tensor)

ref_imgs_rgb = torch.cat(ref_imgs_rgb, dim=0)
gsir_imgs_rgb = torch.cat(gsir_imgs_rgb, dim=0)
tensoir_imgs_rgb = torch.cat(tensoir_imgs_rgb, dim=0)
gtv_imgs_rgb = torch.cat(gtv_imgs_rgb, dim=0)
mine_imgs_rgb = torch.cat(mine_imgs_rgb, dim=0)

Processing image 0...
Processing image 1...
Processing image 2...
Processing image 3...
Processing image 4...
Processing image 5...
Processing image 6...
Processing image 7...
Processing image 8...
Processing image 9...
Processing image 10...
Processing image 11...
Processing image 12...
Processing image 13...
Processing image 14...
Processing image 15...
Processing image 16...
Processing image 17...
Processing image 18...
Processing image 19...
Processing image 20...
Processing image 21...
Processing image 22...
Processing image 23...
Processing image 24...
Processing image 25...
Processing image 26...
Processing image 27...
Processing image 28...
Processing image 29...
Processing image 30...
Processing image 31...
Processing image 32...
Processing image 33...
Processing image 34...
Processing image 35...
Processing image 36...
Processing image 37...
Processing image 38...
Processing image 39...
Processing image 40...
Processing image 41...
Processing image 42...
Processing image 43..

### Alex

In [17]:
# Compute LPIPS
lpips_alex_score0_rgb = lpips_model_alex(ref_imgs_rgb, ref_imgs_rgb)
lpips_alex_score1_rgb = lpips_model_alex(ref_imgs_rgb, gsir_imgs_rgb)
lpips_alex_score2_rgb = lpips_model_alex(ref_imgs_rgb, tensoir_imgs_rgb)
lpips_alex_score3_rgb = lpips_model_alex(ref_imgs_rgb, gtv_imgs_rgb)
lpips_alex_score4_rgb = lpips_model_alex(ref_imgs_rgb, mine_imgs_rgb)

In [18]:
lpips_alex_score0_rgb_mean = torch.mean(lpips_alex_score0_rgb)
lpips_alex_score1_rgb_mean = torch.mean(lpips_alex_score1_rgb)
lpips_alex_score2_rgb_mean = torch.mean(lpips_alex_score2_rgb)
lpips_alex_score3_rgb_mean = torch.mean(lpips_alex_score3_rgb)
lpips_alex_score4_rgb_mean = torch.mean(lpips_alex_score4_rgb)

print(f'Sanity Check: {lpips_alex_score0_rgb_mean}')
print('LPIPS distance between Reference and GS-IR:', lpips_alex_score1_rgb_mean)
print('LPIPS distance between Reference and TensoIR:', lpips_alex_score2_rgb_mean)
print('LPIPS distance between Reference and GTV:', lpips_alex_score3_rgb_mean)
print('LPIPS distance between Reference and Mine:', lpips_alex_score4_rgb_mean)

Sanity Check: 0.0
LPIPS distance between Reference and GS-IR: tensor(0.0805, grad_fn=<MeanBackward0>)
LPIPS distance between Reference and TensoIR: tensor(0.0783, grad_fn=<MeanBackward0>)
LPIPS distance between Reference and GTV: tensor(0.0315, grad_fn=<MeanBackward0>)
LPIPS distance between Reference and Mine: tensor(0.0862, grad_fn=<MeanBackward0>)


### VGG

In [19]:
ref_rgb_score_vgg = []
gsir_rgb_score_vgg = []
tensoir_rgb_score_vgg = []
gtv_rgb_score_vgg = []
mine_rgb_score_vgg = []

In [20]:
interval = 10
for i in range(0, number_of_imgs, interval):
    print(f'Processing images {i} to {i+interval}...')
    sub_batch_ref = ref_imgs_rgb[i:i+interval]
    sub_batch_gsir = gsir_imgs_rgb[i:i+interval]
    sub_batch_tensoir = tensoir_imgs_rgb[i:i+interval]
    sub_batch_gtv = gtv_imgs_rgb[i:i+interval]
    sub_batch_mine = mine_imgs_rgb[i:i+interval]

    with torch.no_grad():
        lpips_vgg_score0 = lpips_model_vgg(sub_batch_ref, sub_batch_ref)
        lpips_vgg_score1 = lpips_model_vgg(sub_batch_ref, sub_batch_gsir)
        lpips_vgg_score2 = lpips_model_vgg(sub_batch_ref, sub_batch_tensoir)
        lpips_vgg_score3 = lpips_model_vgg(sub_batch_ref, sub_batch_gtv)
        lpips_vgg_score4 = lpips_model_vgg(sub_batch_ref, sub_batch_mine)

    ref_rgb_score_vgg.append(lpips_vgg_score0)
    gsir_rgb_score_vgg.append(lpips_vgg_score1)
    tensoir_rgb_score_vgg.append(lpips_vgg_score2)
    gtv_rgb_score_vgg.append(lpips_vgg_score3)
    mine_rgb_score_vgg.append(lpips_vgg_score4)

ref_rgb_score_vgg = torch.cat(ref_rgb_score_vgg).squeeze()
gsir_rgb_score_vgg = torch.cat(gsir_rgb_score_vgg).squeeze()
tensoir_rgb_score_vgg = torch.cat(tensoir_rgb_score_vgg).squeeze()
gtv_rgb_score_vgg = torch.cat(gtv_rgb_score_vgg).squeeze()
mine_rgb_score_vgg = torch.cat(mine_rgb_score_vgg).squeeze()

Processing images 0 to 10...
Processing images 10 to 20...
Processing images 20 to 30...
Processing images 30 to 40...
Processing images 40 to 50...
Processing images 50 to 60...
Processing images 60 to 70...
Processing images 70 to 80...
Processing images 80 to 90...
Processing images 90 to 100...
Processing images 100 to 110...
Processing images 110 to 120...
Processing images 120 to 130...
Processing images 130 to 140...
Processing images 140 to 150...
Processing images 150 to 160...
Processing images 160 to 170...
Processing images 170 to 180...
Processing images 180 to 190...
Processing images 190 to 200...


In [21]:
lpips_vgg_score0_rgb_mean = torch.mean(ref_rgb_score_vgg)
lpips_vgg_score1_rgb_mean = torch.mean(gsir_rgb_score_vgg)
lpips_vgg_score2_rgb_mean = torch.mean(tensoir_rgb_score_vgg)
lpips_vgg_score3_rgb_mean = torch.mean(gtv_rgb_score_vgg)
lpips_vgg_score4_rgb_mean = torch.mean(mine_rgb_score_vgg)

print(f'Sanity Check: {lpips_vgg_score0_rgb_mean}')
print('LPIPS distance between Reference and GS-IR:', lpips_vgg_score1_rgb_mean)
print('LPIPS distance between Reference and TensoIR:', lpips_vgg_score2_rgb_mean)
print('LPIPS distance between Reference and GTV:', lpips_vgg_score3_rgb_mean)
print('LPIPS distance between Reference and Mine:', lpips_vgg_score4_rgb_mean)

Sanity Check: 0.0
LPIPS distance between Reference and GS-IR: tensor(0.0737)
LPIPS distance between Reference and TensoIR: tensor(0.0733)
LPIPS distance between Reference and GTV: tensor(0.0427)
LPIPS distance between Reference and Mine: tensor(0.0649)


### Squeeze

In [22]:
ref_rgb_score_squeeze = []
gsir_rgb_score_squeeze = []
tensoir_rgb_score_squeeze = []
gtv_rgb_score_squeeze = []
mine_rgb_score_squeeze = []

In [23]:
interval = 10
for i in range(0, number_of_imgs, interval):
    print(f'Processing images {i} to {i+interval}...')
    sub_batch_ref = ref_imgs_rgb[i:i+interval]
    sub_batch_gsir = gsir_imgs_rgb[i:i+interval]
    sub_batch_tensoir = tensoir_imgs_rgb[i:i+interval]
    sub_batch_gtv = gtv_imgs_rgb[i:i+interval]
    sub_batch_mine = mine_imgs_rgb[i:i+interval]

    with torch.no_grad():
        lpips_squeeze_score0 = lpips_model_squeeze(sub_batch_ref, sub_batch_ref)
        lpips_squeeze_score1 = lpips_model_squeeze(sub_batch_ref, sub_batch_gsir)
        lpips_squeeze_score2 = lpips_model_squeeze(sub_batch_ref, sub_batch_tensoir)
        lpips_squeeze_score3 = lpips_model_squeeze(sub_batch_ref, sub_batch_gtv)
        lpips_squeeze_score4 = lpips_model_squeeze(sub_batch_ref, sub_batch_mine)

    ref_rgb_score_squeeze.append(lpips_squeeze_score0)
    gsir_rgb_score_squeeze.append(lpips_squeeze_score1)
    tensoir_rgb_score_squeeze.append(lpips_squeeze_score2)
    gtv_rgb_score_squeeze.append(lpips_squeeze_score3)
    mine_rgb_score_squeeze.append(lpips_squeeze_score4)

ref_rgb_score_squeeze = torch.cat(ref_rgb_score_squeeze).squeeze()
gsir_rgb_score_squeeze = torch.cat(gsir_rgb_score_squeeze).squeeze()
tensoir_rgb_score_squeeze = torch.cat(tensoir_rgb_score_squeeze).squeeze()
gtv_rgb_score_squeeze = torch.cat(gtv_rgb_score_squeeze).squeeze()
mine_rgb_score_squeeze = torch.cat(mine_rgb_score_squeeze).squeeze()

Processing images 0 to 10...
Processing images 10 to 20...
Processing images 20 to 30...
Processing images 30 to 40...
Processing images 40 to 50...
Processing images 50 to 60...
Processing images 60 to 70...
Processing images 70 to 80...
Processing images 80 to 90...
Processing images 90 to 100...
Processing images 100 to 110...
Processing images 110 to 120...
Processing images 120 to 130...
Processing images 130 to 140...
Processing images 140 to 150...
Processing images 150 to 160...
Processing images 160 to 170...
Processing images 170 to 180...
Processing images 180 to 190...
Processing images 190 to 200...


In [24]:
lpips_squeeze_score0_rgb_mean = torch.mean(ref_rgb_score_squeeze)
lpips_squeeze_score1_rgb_mean = torch.mean(gsir_rgb_score_squeeze)
lpips_squeeze_score2_rgb_mean = torch.mean(tensoir_rgb_score_squeeze)
lpips_squeeze_score3_rgb_mean = torch.mean(gtv_rgb_score_squeeze)
lpips_squeeze_score4_rgb_mean = torch.mean(mine_rgb_score_squeeze)

print(f'Sanity Check: {lpips_squeeze_score0_rgb_mean}')
print('LPIPS distance between Reference and GS-IR:', lpips_squeeze_score1_rgb_mean)
print('LPIPS distance between Reference and TensoIR:', lpips_squeeze_score2_rgb_mean)
print('LPIPS distance between Reference and GTV:', lpips_squeeze_score3_rgb_mean)
print('LPIPS distance between Reference and Mine:', lpips_squeeze_score4_rgb_mean)

Sanity Check: 0.0
LPIPS distance between Reference and GS-IR: tensor(0.0480)
LPIPS distance between Reference and TensoIR: tensor(0.0525)
LPIPS distance between Reference and GTV: tensor(0.0181)
LPIPS distance between Reference and Mine: tensor(0.0319)


## Fourier Domain

In [25]:
def normalise_fourier_for_lpips(image):
    # Normalise the image to the range [-1, 1], as expected by LPIPS
    img_max = image.max()
    img_min = image.min()
    return (2 * ((image - img_min) / (img_max - img_min)) - 1).astype(np.float32)

def make_rgb_from_grayscale(image):
    # Convert the grayscale image to RGB
    return np.stack((image,) * 3, axis=-1)

In [26]:
ref_imgs_fourier = []
gsir_imgs_fourier = []
tensoir_imgs_fourier = []
gtv_imgs_fourier = []
mine_imgs_fourier = []

for i in range(number_of_imgs):
    print(f'Processing image {i}...')
    # Load the images as grayscale
    ref_gray = cv2.imread(os.path.join(imgs_dir, 'ref', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gsir_gray = cv2.imread(os.path.join(imgs_dir, 'gsir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    tensoir_gray = cv2.imread(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gtv_gray = cv2.imread(os.path.join(imgs_dir, 'mine_gt', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    mine_gray = cv2.imread(os.path.join(imgs_dir, 'mine', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)

    ref_spectrum = make_rgb_from_grayscale(normalise_fourier_for_lpips(compute_fft(ref_gray)))
    gsir_spectrum = make_rgb_from_grayscale(normalise_fourier_for_lpips(compute_fft(gsir_gray)))
    tensoir_spectrum = make_rgb_from_grayscale(normalise_fourier_for_lpips(compute_fft(tensoir_gray)))
    gtv_spectrum = make_rgb_from_grayscale(normalise_fourier_for_lpips(compute_fft(gtv_gray)))
    mine_spectrum = make_rgb_from_grayscale(normalise_fourier_for_lpips(compute_fft(mine_gray)))

    ref_spectrum_tensor = torch.from_numpy(ref_spectrum).permute(2, 0, 1).unsqueeze(0)
    gsir_spectrum_tensor = torch.from_numpy(gsir_spectrum).permute(2, 0, 1).unsqueeze(0)
    tensoir_spectrum_tensor = torch.from_numpy(tensoir_spectrum).permute(2, 0, 1).unsqueeze(0)
    gtv_spectrum_tensor = torch.from_numpy(gtv_spectrum).permute(2, 0, 1).unsqueeze(0)
    mine_spectrum_tensor = torch.from_numpy(mine_spectrum).permute(2, 0, 1).unsqueeze(0)

    ref_imgs_fourier.append(ref_spectrum_tensor)
    gsir_imgs_fourier.append(gsir_spectrum_tensor)
    tensoir_imgs_fourier.append(tensoir_spectrum_tensor)
    gtv_imgs_fourier.append(gtv_spectrum_tensor)
    mine_imgs_fourier.append(mine_spectrum_tensor)

ref_imgs_fourier = torch.cat(ref_imgs_fourier, dim=0)
gsir_imgs_fourier = torch.cat(gsir_imgs_fourier, dim=0)
tensoir_imgs_fourier = torch.cat(tensoir_imgs_fourier, dim=0)
gtv_imgs_fourier = torch.cat(gtv_imgs_fourier, dim=0)
mine_imgs_fourier = torch.cat(mine_imgs_fourier, dim=0)

Processing image 0...
Processing image 1...
Processing image 2...
Processing image 3...
Processing image 4...
Processing image 5...
Processing image 6...
Processing image 7...
Processing image 8...
Processing image 9...
Processing image 10...
Processing image 11...
Processing image 12...
Processing image 13...
Processing image 14...
Processing image 15...
Processing image 16...
Processing image 17...
Processing image 18...
Processing image 19...
Processing image 20...
Processing image 21...
Processing image 22...
Processing image 23...
Processing image 24...
Processing image 25...
Processing image 26...
Processing image 27...
Processing image 28...
Processing image 29...
Processing image 30...
Processing image 31...
Processing image 32...
Processing image 33...
Processing image 34...
Processing image 35...
Processing image 36...
Processing image 37...
Processing image 38...
Processing image 39...
Processing image 40...
Processing image 41...
Processing image 42...
Processing image 43..

### Alex

In [27]:
lpips_alex_score0_fourier = lpips_model_alex(ref_imgs_fourier, ref_imgs_fourier)
lpips_alex_score1_fourier = lpips_model_alex(ref_imgs_fourier, gsir_imgs_fourier)
lpips_alex_score2_fourier = lpips_model_alex(ref_imgs_fourier, tensoir_imgs_fourier)
lpips_alex_score3_fourier = lpips_model_alex(ref_imgs_fourier, gtv_imgs_fourier)
lpips_alex_score4_fourier = lpips_model_alex(ref_imgs_fourier, mine_imgs_fourier)

In [28]:
lpips_alex_score0_fourier_mean = torch.mean(lpips_alex_score0_fourier)
lpips_alex_score1_fourier_mean = torch.mean(lpips_alex_score1_fourier)
lpips_alex_score2_fourier_mean = torch.mean(lpips_alex_score2_fourier)
lpips_alex_score3_fourier_mean = torch.mean(lpips_alex_score3_fourier)
lpips_alex_score4_fourier_mean = torch.mean(lpips_alex_score4_fourier)

print(f'Sanity Check: {lpips_alex_score0_fourier_mean}')
print('LPIPS distance between Reference and GS-IR:', lpips_alex_score1_fourier_mean)
print('LPIPS distance between Reference and TensoIR:', lpips_alex_score2_fourier_mean)
print('LPIPS distance between Reference and GtV:', lpips_alex_score3_fourier_mean)
print('LPIPS distance between Reference and Mine:', lpips_alex_score4_fourier_mean)

Sanity Check: 0.0
LPIPS distance between Reference and GS-IR: tensor(0.0002, grad_fn=<MeanBackward0>)
LPIPS distance between Reference and TensoIR: tensor(0.0002, grad_fn=<MeanBackward0>)
LPIPS distance between Reference and GtV: tensor(2.7474e-05, grad_fn=<MeanBackward0>)
LPIPS distance between Reference and Mine: tensor(0.0001, grad_fn=<MeanBackward0>)


### VGG

In [29]:
ref_fourier_score_vgg = []
gsir_fourier_score_vgg = []
tensoir_fourier_score_vgg = []
gtv_fourier_score_vgg = []
mine_fourier_score_vgg = []

In [30]:
interval = 10
for i in range(0, number_of_imgs, interval):
    print(f'Processing images {i} to {i+interval}...')
    sub_batch_ref = ref_imgs_fourier[i:i+interval]
    sub_batch_gsir = gsir_imgs_fourier[i:i+interval]
    sub_batch_tensoir = tensoir_imgs_fourier[i:i+interval]
    sub_batch_gtv = gtv_imgs_fourier[i:i+interval]
    sub_batch_mine = mine_imgs_fourier[i:i+interval]

    with torch.no_grad():
        lpips_vgg_score0 = lpips_model_vgg(sub_batch_ref, sub_batch_ref)
        lpips_vgg_score1 = lpips_model_vgg(sub_batch_ref, sub_batch_gsir)
        lpips_vgg_score2 = lpips_model_vgg(sub_batch_ref, sub_batch_tensoir)
        lpips_vgg_score3 = lpips_model_vgg(sub_batch_ref, sub_batch_gtv)
        lpips_vgg_score4 = lpips_model_vgg(sub_batch_ref, sub_batch_mine)

    ref_fourier_score_vgg.append(lpips_vgg_score0)
    gsir_fourier_score_vgg.append(lpips_vgg_score1)
    tensoir_fourier_score_vgg.append(lpips_vgg_score2)
    gtv_fourier_score_vgg.append(lpips_vgg_score3)
    mine_fourier_score_vgg.append(lpips_vgg_score4)

ref_fourier_score_vgg = torch.cat(ref_fourier_score_vgg).squeeze()
gsir_fourier_score_vgg = torch.cat(gsir_fourier_score_vgg).squeeze()
tensoir_fourier_score_vgg = torch.cat(tensoir_fourier_score_vgg).squeeze()
gtv_fourier_score_vgg = torch.cat(gtv_fourier_score_vgg).squeeze()
mine_fourier_score_vgg = torch.cat(mine_fourier_score_vgg).squeeze()

Processing images 0 to 10...
Processing images 10 to 20...
Processing images 20 to 30...
Processing images 30 to 40...
Processing images 40 to 50...
Processing images 50 to 60...
Processing images 60 to 70...
Processing images 70 to 80...
Processing images 80 to 90...
Processing images 90 to 100...
Processing images 100 to 110...
Processing images 110 to 120...
Processing images 120 to 130...
Processing images 130 to 140...
Processing images 140 to 150...
Processing images 150 to 160...
Processing images 160 to 170...
Processing images 170 to 180...
Processing images 180 to 190...
Processing images 190 to 200...


In [31]:
lpips_vgg_score0_fourier_mean = torch.mean(ref_fourier_score_vgg)
lpips_vgg_score1_fourier_mean = torch.mean(gsir_fourier_score_vgg)
lpips_vgg_score2_fourier_mean = torch.mean(tensoir_fourier_score_vgg)
lpips_vgg_score3_fourier_mean = torch.mean(gtv_fourier_score_vgg)
lpips_vgg_score4_fourier_mean = torch.mean(mine_fourier_score_vgg)

print(f'Sanity Check: {lpips_vgg_score0_fourier_mean}')
print('LPIPS distance between Reference and GS-IR:', lpips_vgg_score1_fourier_mean)
print('LPIPS distance between Reference and TensoIR:', lpips_vgg_score2_fourier_mean)
print('LPIPS distance between Reference and GtV:', lpips_vgg_score3_fourier_mean)
print('LPIPS distance between Reference and Mine:', lpips_vgg_score4_fourier_mean)

Sanity Check: 0.0
LPIPS distance between Reference and GS-IR: tensor(0.0113)
LPIPS distance between Reference and TensoIR: tensor(0.0109)
LPIPS distance between Reference and GtV: tensor(0.0015)
LPIPS distance between Reference and Mine: tensor(0.0090)


### Squeeze

In [32]:
ref_fourier_score_squeeze = []
gsir_fourier_score_squeeze = []
tensoir_fourier_score_squeeze = []
gtv_fourier_score_squeeze = []
mine_fourier_score_squeeze = []

In [33]:
interval = 10
for i in range(0, number_of_imgs, interval):
    print(f'Processing images {i} to {i+interval}...')
    sub_batch_ref = ref_imgs_fourier[i:i+interval]
    sub_batch_gsir = gsir_imgs_fourier[i:i+interval]
    sub_batch_tensoir = tensoir_imgs_fourier[i:i+interval]
    sub_batch_gtv = gtv_imgs_fourier[i:i+interval]
    sub_batch_mine = mine_imgs_fourier[i:i+interval]

    with torch.no_grad():
        lpips_squeeze_score0 = lpips_model_squeeze(sub_batch_ref, sub_batch_ref)
        lpips_squeeze_score1 = lpips_model_squeeze(sub_batch_ref, sub_batch_gsir)
        lpips_squeeze_score2 = lpips_model_squeeze(sub_batch_ref, sub_batch_tensoir)
        lpips_squeeze_score3 = lpips_model_squeeze(sub_batch_ref, sub_batch_gtv)
        lpips_squeeze_score4 = lpips_model_squeeze(sub_batch_ref, sub_batch_mine)

    ref_fourier_score_squeeze.append(lpips_squeeze_score0)
    gsir_fourier_score_squeeze.append(lpips_squeeze_score1)
    tensoir_fourier_score_squeeze.append(lpips_squeeze_score2)
    gtv_fourier_score_squeeze.append(lpips_squeeze_score3)
    mine_fourier_score_squeeze.append(lpips_squeeze_score4)

ref_fourier_score_squeeze = torch.cat(ref_fourier_score_squeeze).squeeze()
gsir_fourier_score_squeeze = torch.cat(gsir_fourier_score_squeeze).squeeze()
tensoir_fourier_score_squeeze = torch.cat(tensoir_fourier_score_squeeze).squeeze()
gtv_fourier_score_squeeze = torch.cat(gtv_fourier_score_squeeze).squeeze()
mine_fourier_score_squeeze = torch.cat(mine_fourier_score_squeeze).squeeze()

Processing images 0 to 10...
Processing images 10 to 20...
Processing images 20 to 30...
Processing images 30 to 40...
Processing images 40 to 50...
Processing images 50 to 60...
Processing images 60 to 70...
Processing images 70 to 80...
Processing images 80 to 90...
Processing images 90 to 100...
Processing images 100 to 110...
Processing images 110 to 120...
Processing images 120 to 130...
Processing images 130 to 140...
Processing images 140 to 150...
Processing images 150 to 160...
Processing images 160 to 170...
Processing images 170 to 180...
Processing images 180 to 190...
Processing images 190 to 200...


In [34]:
lpips_squeeze_score0_fourier_mean = torch.mean(ref_fourier_score_squeeze)
lpips_squeeze_score1_fourier_mean = torch.mean(gsir_fourier_score_squeeze)
lpips_squeeze_score2_fourier_mean = torch.mean(tensoir_fourier_score_squeeze)
lpips_squeeze_score3_fourier_mean = torch.mean(gtv_fourier_score_squeeze)
lpips_squeeze_score4_fourier_mean = torch.mean(mine_fourier_score_squeeze)

print(f'Sanity Check: {lpips_squeeze_score0_fourier_mean}')
print('LPIPS distance between Reference and GS-IR:', lpips_squeeze_score1_fourier_mean)
print('LPIPS distance between Reference and TensoIR:', lpips_squeeze_score2_fourier_mean)
print('LPIPS distance between Reference and GtV:', lpips_squeeze_score3_fourier_mean)
print('LPIPS distance between Reference and Mine:', lpips_squeeze_score4_fourier_mean)

Sanity Check: 0.0
LPIPS distance between Reference and GS-IR: tensor(0.0014)
LPIPS distance between Reference and TensoIR: tensor(0.0013)
LPIPS distance between Reference and GtV: tensor(0.0001)
LPIPS distance between Reference and Mine: tensor(0.0008)


## FLIP

Check the FLIP folder.